# Данные

В исследовании используются данные с рейтингами управляющих организаций города Москвы за 2019, 2020 и 2021 год.

Данные взяты с сайта https://data-new.mos.ru/.

Описание данных с портала открытых данных Правительства Москвы:

"Мосжилинспекцией ведется анализ работы управляющих организаций города Москвы. По результатам анализа ежегодно составляется рейтинг управляющих организаций. Организации оцениваются по трём основным показателям: количество нарушений, уровень надежности и степень удовлетворения населения. Рейтинг управляющих организаций призван повысить прозрачность и качество оказания услуг в сфере управления многоквартирным жилым фондом. Рейтинг позволяет жителям получать объективную информацию о качестве работы их управляющих организаций, видеть, какое место она занимает среди других управляющих организаций, работающих на территории города Москвы. Это помогает поддерживать здоровую конкуренцию между участниками этого рынка, а жителям - принимать правильные решения, к примеру, о замене неэффективно работающих управляющих организаций на более эффективные."

# Какие данные представлены?

- Локальный идентификатор каждой УО
- Наименование управляющей организации
- ИНН организации
- Административный округ, в котром работает организация
- Количество домов в управлении организации
- Площадь домов в управлении организации (кв. м)
- Уровень удовлетворенности населения
- Уровень надежности УО
- Уровень нарушений в работе УО
- Суммарное количество баллов - Уровень удовлетворенности населения * 0,4 + Уровень надежности УО * 0,2 + Уровень нарушений в работе УО * 0,4
- Итоговый рейтинг
- global_id - глобальный идентификатор УО

# Исследуемые вопросы

1. В каких административных округах находится наибольшее количество компаний? Происходят ли изменения от года к году?

2. Какие компании являются самыми крупными? В каких округах они ведут работу?

3. Являются ли изменения в оценках компаний статистически значимыми?

4. Как менялись оценки 10 самых крупных компаний?

# Предобработка данных

In [2]:
# Загрузка нужных библиотек
import numpy as np
import pandas as pd
import re

from IPython.display import Image, display

from scipy.stats import shapiro
import scipy.stats as stats

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
# Загрузка данных
rate_2019 = pd.read_excel('data/rating_2019.xlsx')
rate_2020 = pd.read_excel('data/rating_2020.xlsx')
rate_2021 = pd.read_excel('data/rating_2021.xlsx')

## 1. Рейтинг компаний за 2019 год

In [4]:
print(rate_2019.shape)
rate_2019.head()

(562, 12)


,Локальный идентификатор,Наименование управляющей организации,ИНН,Административный округ,Количество домов в управлении,Площадь домов в управлении (кв. м),Уровень удовлетворенности населения,Уровень надежности УО,Уровень нарушений в работе УО,"Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4",Итоговый рейтинг,global_id
0,1,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «УПРА...,9706004109,Центральный административный округ,1,5267,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:100\nPenaltyScores:10...,104.00,1,1075032366
1,93,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «КОМП...,7716196822,Северо-Восточный административный округ,32,353030,ScoresOfAppeals:99.11\nCoefficientValue:1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:99.69\nPenaltyScores:...,101.96,93,1075032367
2,26,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «КОНОК»,7709448067,Центральный административный округ,1,8011,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:99.81\nPenaltyScores:...,103.19,26,1075032368
3,27,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «УПРА...,7702660770,Центральный административный округ,8,41964,ScoresOfAppeals:98.52\nCoefficientValue:1.1\nS...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:99.56\nPenaltyScores:...,103.13,27,1075032369
4,2,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «АВРО...,7719778219,Центральный административный округ,1,4531,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:100\nPenaltyScores:10...,104.00,2,1075032370


В рейтинге за 2019 год 562 строки и 12 столбцов.

Оставим только нужные столбцы, затем поменяем названия колонок для удобства работы.

Удалим лишнюю информацию из колонок с названиями компаний и административных округов.

In [5]:
rate_2019 = rate_2019[['Наименование управляющей организации',
                       'Административный округ',
                       'Количество домов в управлении',
                       'Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4',
                       'Уровень удовлетворенности населения',
                       'Уровень надежности УО',
                       'Уровень нарушений в работе УО',
                       'global_id']]

rate_2019 = rate_2019.rename(columns={'Наименование управляющей организации': 'company',
                                      'Административный округ': 'region',
                                      'Количество домов в управлении': 'houses_count',
                                      'Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4': 'points',
                                      'Уровень удовлетворенности населения': 'satisfaction_level',
                                      'Уровень надежности УО': 'reliability_level',
                                      'Уровень нарушений в работе УО': 'violations_level'})

rate_2019['company'] = rate_2019['company'].str.replace('ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ', '') \
                                           .str.replace('ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ', '') \
                                           .str.lstrip()

rate_2019['region'] = rate_2019['region'].str.replace('административный округ', '')

rate_2019.head()

,company,region,houses_count,points,satisfaction_level,reliability_level,violations_level,global_id
0,«УПРАВЛЯЮЩАЯ КОМПАНИЯ ЦАО»,Центральный,1,104.00,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:100\nPenaltyScores:10...,1075032366
1,«КОМПЛЕКСНОЕ ОБСЛУЖИВАНИЕ РАЙОНА»,Северо-Восточный,32,101.96,ScoresOfAppeals:99.11\nCoefficientValue:1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:99.69\nPenaltyScores:...,1075032367
2,«КОНОК»,Центральный,1,103.19,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:99.81\nPenaltyScores:...,1075032368
3,«УПРАВЛЯЮЩАЯ КОМПАНИЯ МЕЩАНСКАЯ СЛОБОДА»,Центральный,8,103.13,ScoresOfAppeals:98.52\nCoefficientValue:1.1\nS...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:99.56\nPenaltyScores:...,1075032369
4,«АВРОРА ПЛЮС»,Центральный,1,104.00,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,ScoresByStandart:100\nScoresSumZ2:100\nInterme...,DetectedViolationsScores:100\nPenaltyScores:10...,1075032370


Видно, что данные с уровнем удовлетворенности населения, уровнем надежности УО и уровнем нарушений в работе УО требуют обработки. Выделим из данных колонок нужную информацию.

In [6]:
def find_score(str_before, str_after, raw_score):
    '''
    Функция производит поиск нужного значения в строке. Известны строка, идущая перед искомым значением и строка, идущая сразу после искомого значения.
    В переменную start_pos записывается индекс начала искомой строки, в переменную end_pos записывается индекс конца искомой строки.
    Переменная score содержит искомое значение.
    '''
    start_pos = re.search(str_before, raw_score).end()
    end_pos = re.search(str_after, raw_score).start()
    score = raw_score[start_pos:end_pos].rstrip()
    return score

In [7]:
rate_2019['satisfaction_level'] = rate_2019['satisfaction_level'].apply(lambda x: find_score('ScoresSumZ1:','Intermediate',x)).astype(float)
rate_2019['reliability_level'] = rate_2019['reliability_level'].apply(lambda x: find_score('ScoresSumZ2:','Intermediate',x)).astype(float)
rate_2019['violations_level'] = rate_2019['violations_level'].apply(lambda x: find_score('ScoresSumZ3:','Intermediate',x)).astype(float)

rate_2019.head()

,company,region,houses_count,points,satisfaction_level,reliability_level,violations_level,global_id
0,«УПРАВЛЯЮЩАЯ КОМПАНИЯ ЦАО»,Центральный,1,104.00,110.00,100.0,100.00,1075032366
1,«КОМПЛЕКСНОЕ ОБСЛУЖИВАНИЕ РАЙОНА»,Северо-Восточный,32,101.96,105.00,100.0,99.85,1075032367
2,«КОНОК»,Центральный,1,103.19,110.00,100.0,97.99,1075032368
3,«УПРАВЛЯЮЩАЯ КОМПАНИЯ МЕЩАНСКАЯ СЛОБОДА»,Центральный,8,103.13,108.37,100.0,99.45,1075032369
4,«АВРОРА ПЛЮС»,Центральный,1,104.00,110.00,100.0,100.00,1075032370


Посмотрим, нет ли повторяющихся идентификаторов компаний?

Повторов нет, так как число уникальных global_id равно числу строк датафрейма.

In [8]:
rate_2019['global_id'].nunique() == rate_2019.shape[0]

True

Посмотрим, сколько компаний действуют внутри каждого административного округа.

Также посмотрим на среднее и медианное значение оценок компаний.

Средние и медианы отличаются не слишком сильно, это говорит о том, что в данных внутри каждого административного округа нет сильных выбросов.

In [9]:
agg_by_region_2019 = rate_2019.groupby('region')['points'] \
                              .agg(['count', 'mean', 'median']) \
                              .sort_values(by='count', ascending=False) \
                              .head(10) \
                              .reset_index()

agg_by_region_2019

,region,count,mean,median
0,Центральный,85,98.202235,100.800
1,Северный,62,99.172258,100.885
2,Новомосковский,51,98.883529,99.940
3,Восточный,49,101.121633,101.430
4,Северо-Восточный,47,98.925532,100.640
5,Западный,45,99.313111,100.410
6,Юго-Западный,39,98.721538,100.680
7,Юго-Восточный,38,100.021842,100.000
8,Южный,33,99.822727,100.980
9,Северо-Западный,25,98.324400,99.970


Выделим 10 самых крупных компаний (с наибольшим количеством домов в управлении).

Видим, что есть компания, работающая сразу в нескольких административных округах. В управлении этой компании также находится наибольшее количество домов.

In [10]:
sorted_by_house_count_2019 = rate_2019.sort_values(by='houses_count', ascending=False).head(10)[['company', 'region', 'houses_count']].reset_index(drop=True)

sorted_by_house_count_2019

,company,region,houses_count
0,«ПИК-КОМФОРТ»,"[Центральный , Северный , Северо-Восточный , Ю...",534
1,«ЖИЛИЩНИК ПРЕСНЕНСКОГО РАЙОНА»,Центральный,512
2,«ЖИЛИЩНИК БАСМАННОГО РАЙОНА»,Центральный,496
3,«ЖИЛИЩНИК РАЙОНА ТВЕРСКОЙ»,Центральный,427
4,«ЖИЛИЩНИК РАЙОНА ХОРОШЕВО-МНЕВНИКИ»,Северо-Западный,423
5,«ЖИЛИЩНИК РАЙОНА ЛЮБЛИНО»,Юго-Восточный,416
6,«ЖИЛИЩНИК МОЖАЙСКОГО РАЙОНА»,Западный,388
7,«ЖИЛИЩНИК РАЙОНА ЮЖНОЕ БУТОВО»,Юго-Западный,383
8,«ЖИЛИЩНИК РАЙОНА КУНЦЕВО»,Западный,363
9,«ЖИЛИЩНИК ТАГАНСКОГО РАЙОНА»,Центральный,360


## 2. Рейтинг компаний за 2020 год

In [11]:
print(rate_2020.shape)
rate_2020.head()

(605, 12)


,Локальный идентификатор,Наименование управляющей организации,ИНН,Административный округ,Количество домов в управлении,Площадь домов в управлении (кв. м),Уровень удовлетворенности населения,Уровень надежности УО,Уровень нарушений в работе УО,"Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4",Итоговый рейтинг,global_id
0,13127,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «УК В...,7734402644,Северо-Западный административный округ,5,33976,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.78\nScoresSumZ2:0.78\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,84.16,3,1719509048
1,13128,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «ЦЕНТ...,7733800995,Восточный административный округ,1,10989,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:1\nScoresSumZ2:1\nIntermediat...,DetectedViolationsScores:100\nPenaltyScores:10...,84.20,1,1721300782
2,13129,АКЦИОНЕРНОЕ ОБЩЕСТВО «ВПЕРЁД ДВЛ»,7714436148,Центральный административный округ,1,3698,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.98\nScoresSumZ2:0.98\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,84.20,2,1721300783
3,13130,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «АКВАНТ»,1831005666,Северный административный округ,1,13439,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.81\nScoresSumZ2:0.81\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,84.16,4,1721300784
4,13131,ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ГОРОДА М...,7719005931,Восточный административный округ,1,6291,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0\nScoresSumZ2:0\nIntermediat...,DetectedViolationsScores:100\nPenaltyScores:10...,84.00,5,1721300785


В рейтинге за 2020 год 605 строки и 12 столбцов.

Оставим только нужные столбцы, затем поменяем названия колонок для удобства работы.

Удалим лишнюю информацию из колонок с названиями компаний и административных округов.

In [12]:
rate_2020 = rate_2020[['Наименование управляющей организации',
                       'Административный округ',
                       'Количество домов в управлении',
                       'Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4',
                       'Уровень удовлетворенности населения',
                       'Уровень надежности УО',
                       'Уровень нарушений в работе УО',
                       'global_id']]

rate_2020 = rate_2020.rename(columns={'Наименование управляющей организации': 'company',
                                      'Административный округ': 'region',
                                      'Количество домов в управлении': 'houses_count',
                                      'Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4': 'points',
                                      'Уровень удовлетворенности населения': 'satisfaction_level',
                                      'Уровень надежности УО': 'reliability_level',
                                      'Уровень нарушений в работе УО': 'violations_level'})

rate_2020['company'] = rate_2020['company'].str.replace('ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ', '') \
                                           .str.replace('ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ', '') \
                                           .str.lstrip()

rate_2020['region'] = rate_2020['region'].str.replace('административный округ', '')

rate_2020.head()

,company,region,houses_count,points,satisfaction_level,reliability_level,violations_level,global_id
0,«УК ВОСТОК ГРУПП»,Северо-Западный,5,84.16,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.78\nScoresSumZ2:0.78\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,1719509048
1,«ЦЕНТР СЕРВИСНОГО ОБСЛУЖИВАНИЯ ЭНТУЗИАСТ»,Восточный,1,84.20,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:1\nScoresSumZ2:1\nIntermediat...,DetectedViolationsScores:100\nPenaltyScores:10...,1721300782
2,АКЦИОНЕРНОЕ ОБЩЕСТВО «ВПЕРЁД ДВЛ»,Центральный,1,84.20,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.98\nScoresSumZ2:0.98\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,1721300783
3,«АКВАНТ»,Северный,1,84.16,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.81\nScoresSumZ2:0.81\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,1721300784
4,ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ГОРОДА М...,Восточный,1,84.00,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0\nScoresSumZ2:0\nIntermediat...,DetectedViolationsScores:100\nPenaltyScores:10...,1721300785


Видно, что данные с уровнем удовлетворенности населения, уровнем надежности УО и уровнем нарушений в работе УО требуют обработки. Выделим из данных колонок нужную информацию.

In [13]:
rate_2020['satisfaction_level'] = rate_2020['satisfaction_level'].apply(lambda x: find_score('ScoresSumZ1:','Intermediate',x)).astype(float)
rate_2020['reliability_level'] = rate_2020['reliability_level'].apply(lambda x: find_score('ScoresSumZ2:','Intermediate',x)).astype(float) * 100
rate_2020['violations_level'] = rate_2020['violations_level'].apply(lambda x: find_score('ScoresSumZ3:','Intermediate',x)).astype(float)

rate_2020.head()

,company,region,houses_count,points,satisfaction_level,reliability_level,violations_level,global_id
0,«УК ВОСТОК ГРУПП»,Северо-Западный,5,84.16,110.0,78.0,100.0,1719509048
1,«ЦЕНТР СЕРВИСНОГО ОБСЛУЖИВАНИЯ ЭНТУЗИАСТ»,Восточный,1,84.20,110.0,100.0,100.0,1721300782
2,АКЦИОНЕРНОЕ ОБЩЕСТВО «ВПЕРЁД ДВЛ»,Центральный,1,84.20,110.0,98.0,100.0,1721300783
3,«АКВАНТ»,Северный,1,84.16,110.0,81.0,100.0,1721300784
4,ГОСУДАРСТВЕННОЕ УНИТАРНОЕ ПРЕДПРИЯТИЕ ГОРОДА М...,Восточный,1,84.00,110.0,0.0,100.0,1721300785


Посмотрим, нет ли повторяющихся идентификаторов компаний?

Повторов нет, так как число уникальных global_id равно числу строк датафрейма.

In [14]:
rate_2020['global_id'].nunique() == rate_2020.shape[0]

True

Посмотрим, сколько компаний действуют внутри каждого административного округа.

Также посмотрим на среднее и медианное значение оценок компаний.

Средние и медианы отличаются не слишком сильно, это говорит о том, что в данных внутри каждого административного округа нет сильных выбросов.

In [15]:
agg_by_region_2020 = rate_2020.groupby('region')['points'] \
                              .agg(['count', 'mean', 'median']) \
                              .sort_values(by='count', ascending=False) \
                              .head(10) \
                              .reset_index()
agg_by_region_2020

,region,count,mean,median
0,Центральный,86,79.259884,80.185
1,Троицкий,83,79.223735,79.930
2,Северный,61,76.446393,76.490
3,Восточный,52,77.414038,77.365
4,Западный,50,77.825000,79.870
5,Северо-Восточный,47,73.186809,74.590
6,Юго-Западный,40,78.690500,79.890
7,Юго-Восточный,39,74.112821,75.560
8,Южный,35,77.860571,78.340
9,Северо-Западный,29,77.715862,77.970


Выделим 10 самых крупных компаний (с наибольшим количеством домов в управлении).

Видим, что есть компания, работающая сразу в нескольких административных округах. В управлении этой компании также находится наибольшее количество домов.

In [16]:
sorted_by_house_count_2020 = rate_2020.sort_values(by='houses_count', ascending=False).head(10)[['company', 'region', 'houses_count']].reset_index(drop=True)

sorted_by_house_count_2020

,company,region,houses_count
0,«ПИК-КОМФОРТ»,"[Восточный , Западный , Зеленоградский , Троиц...",531
1,«ЖИЛИЩНИК ПРЕСНЕНСКОГО РАЙОНА»,Центральный,518
2,«ЖИЛИЩНИК БАСМАННОГО РАЙОНА»,Центральный,504
3,«ЖИЛИЩНИК ЗЕЛЕНОГРАДСКОГО АДМИНИСТРАТИВНОГО ОК...,Зеленоградский,477
4,«ЖИЛИЩНИК РАЙОНА ТВЕРСКОЙ»,Центральный,432
5,«ЖИЛИЩНИК РАЙОНА ХОРОШЕВО-МНЕВНИКИ»,Северо-Западный,423
6,«ЖИЛИЩНИК РАЙОНА ЛЮБЛИНО»,Юго-Восточный,416
7,«ЖИЛИЩНИК МОЖАЙСКОГО РАЙОНА»,Западный,389
8,«ЖИЛИЩНИК РАЙОНА ЮЖНОЕ БУТОВО»,Юго-Западный,379
9,«ЖИЛИЩНИК ТАГАНСКОГО РАЙОНА»,Центральный,361


## 3. Рейтинг компаний за 2021 год

In [17]:
print(rate_2021.shape)
rate_2021.head()

(622, 12)


,Локальный идентификатор,Наименование управляющей организации,ИНН,Административный округ,Количество домов в управлении,Площадь домов в управлении (кв. м),Уровень удовлетворенности населения,Уровень надежности УО,Уровень нарушений в работе УО,"Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4",Итоговый рейтинг,global_id
0,11529,АКЦИОНЕРНОЕ ОБЩЕСТВО «СТРОЙСЕРВИС»,7710034550,Центральный административный округ,2,27872,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.94\nScoresSumZ2:0.94\nInter...,DetectedViolationsScores:99.89\nPenaltyScores:...,84.17,1,2387463246
1,11530,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «РУСГ...,7720483509,Центральный административный округ,1,15806,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:1\nScoresSumZ2:1\nIntermediat...,DetectedViolationsScores:99.81\nPenaltyScores:...,84.17,2,2387463247
2,11531,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «ГОРО...,7743247071,Центральный административный округ,1,348,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.67\nScoresSumZ2:0.67\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,84.13,3,2387463248
3,11532,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «ПОЛЕ...,7703385090,Центральный административный округ,1,973,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.62\nScoresSumZ2:0.62\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,84.12,4,2387463249
4,11533,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ «АРСП»,7743085906,Северо-Западный административный округ,1,4099,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:1\nScoresSumZ2:1\nIntermediat...,DetectedViolationsScores:99.28\nPenaltyScores:...,84.00,5,2387463250


В рейтинге за 2021 год 622 строки и 12 столбцов.

Оставим только нужные столбцы, затем поменяем названия колонок для удобства работы.

Удалим лишнюю информацию из колонок с названиями компаний и административных округов.

In [18]:
rate_2021 = rate_2021[['Наименование управляющей организации',
                       'Административный округ',
                       'Количество домов в управлении',
                       'Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4',
                       'Уровень удовлетворенности населения',
                       'Уровень надежности УО',
                       'Уровень нарушений в работе УО',
                       'global_id']]

rate_2021 = rate_2021.rename(columns={'Наименование управляющей организации': 'company',
                                      'Административный округ': 'region',
                                      'Количество домов в управлении': 'houses_count',
                                      'Суммарное количество баллов Z1*0,4+Z2*0,2+Z3*0,4': 'points',
                                      'Уровень удовлетворенности населения': 'satisfaction_level',
                                      'Уровень надежности УО': 'reliability_level',
                                      'Уровень нарушений в работе УО': 'violations_level'})

rate_2021['company'] = rate_2021['company'].str.replace('ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ', '') \
                                           .str.replace('ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МОСКВЫ', '') \
                                           .str.lstrip()

rate_2021['region'] = rate_2021['region'].str.replace('административный округ', '')

rate_2021.head()

,company,region,houses_count,points,satisfaction_level,reliability_level,violations_level,global_id
0,АКЦИОНЕРНОЕ ОБЩЕСТВО «СТРОЙСЕРВИС»,Центральный,2,84.17,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.94\nScoresSumZ2:0.94\nInter...,DetectedViolationsScores:99.89\nPenaltyScores:...,2387463246
1,«РУСГОРОД»,Центральный,1,84.17,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:1\nScoresSumZ2:1\nIntermediat...,DetectedViolationsScores:99.81\nPenaltyScores:...,2387463247
2,«ГОРОДСКАЯ СЛУЖБА ЭКСПЛУАТАЦИИ»,Центральный,1,84.13,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.67\nScoresSumZ2:0.67\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,2387463248
3,«ПОЛЕТ-К»,Центральный,1,84.12,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:0.62\nScoresSumZ2:0.62\nInter...,DetectedViolationsScores:100\nPenaltyScores:10...,2387463249
4,«АРСП»,Северо-Западный,1,84.00,ScoresOfAppeals:100\nCoefficientValue:1.1\nSco...,DisclosureScores:1\nScoresSumZ2:1\nIntermediat...,DetectedViolationsScores:99.28\nPenaltyScores:...,2387463250


Видно, что данные с уровнем удовлетворенности населения, уровнем надежности УО и уровнем нарушений в работе УО требуют обработки. Выделим из данных колонок нужную информацию.

In [19]:
rate_2021['satisfaction_level'] = rate_2021['satisfaction_level'].apply(lambda x: find_score('ScoresSumZ1:','Intermediate',x)).astype(float)
rate_2021['reliability_level'] = rate_2021['reliability_level'].apply(lambda x: find_score('ScoresSumZ2:','Intermediate',x)).astype(float) * 100
rate_2021['violations_level'] = rate_2021['violations_level'].apply(lambda x: find_score('ScoresSumZ3:','Intermediate',x)).astype(float)

rate_2021.head()

,company,region,houses_count,points,satisfaction_level,reliability_level,violations_level,global_id
0,АКЦИОНЕРНОЕ ОБЩЕСТВО «СТРОЙСЕРВИС»,Центральный,2,84.17,110.0,94.0,99.96,2387463246
1,«РУСГОРОД»,Центральный,1,84.17,110.0,100.0,99.92,2387463247
2,«ГОРОДСКАЯ СЛУЖБА ЭКСПЛУАТАЦИИ»,Центральный,1,84.13,110.0,67.0,100.00,2387463248
3,«ПОЛЕТ-К»,Центральный,1,84.12,110.0,62.0,100.00,2387463249
4,«АРСП»,Северо-Западный,1,84.00,110.0,100.0,99.71,2387463250


Посмотрим, нет ли повторяющихся идентификаторов компаний?

Повторов нет, так как число уникальных global_id равно числу строк датафрейма.

In [20]:
rate_2021['global_id'].nunique() == rate_2021.shape[0]

True

Посмотрим, сколько компаний действуют внутри каждого административного округа.

Также посмотрим на среднее и медианное значение оценок компаний.

Средние и медианы отличаются не слишком сильно, это говорит о том, что в данных внутри каждого административного округа нет сильных выбросов.

In [21]:
agg_by_region_2021 = rate_2021.groupby('region')['points'] \
                              .agg(['count', 'mean', 'median']) \
                              .sort_values(by='count', ascending=False) \
                              .head(10) \
                              .reset_index()
agg_by_region_2021

,region,count,mean,median
0,Центральный,86,81.058372,80.955
1,"[Новомосковский , Троицкий ]",81,80.197160,79.970
2,Северный,60,80.758500,80.725
3,Западный,52,80.242500,80.195
4,Восточный,52,79.585000,80.780
5,Северо-Восточный,47,80.262128,80.430
6,Юго-Западный,41,80.731707,80.530
7,Юго-Восточный,39,80.171026,80.100
8,Южный,38,80.347368,80.380
9,Северо-Западный,30,80.216667,80.380


Выделим 10 самых крупных компаний (с наибольшим количеством домов в управлении).

Видим, что есть компания, работающая сразу в нескольких административных округах. В управлении этой компании также находится наибольшее количество домов.

In [22]:
sorted_by_house_count_2021 = rate_2021.sort_values(by='houses_count', ascending=False).head(10)[['company', 'region', 'houses_count']].reset_index(drop=True)

sorted_by_house_count_2021

,company,region,houses_count
0,«ПИК-КОМФОРТ»,"[Восточный , Западный , Зеленоградский , Север...",537
1,«ЖИЛИЩНИК РАЙОНА ПЕРОВО»,Восточный,529
2,«ЖИЛИЩНИК ПРЕСНЕНСКОГО РАЙОНА»,Центральный,523
3,«ЖИЛИЩНИК БАСМАННОГО РАЙОНА»,Центральный,505
4,«ЖИЛИЩНИК ЗЕЛЕНОГРАДСКОГО АДМИНИСТРАТИВНОГО ОК...,Зеленоградский,485
5,«ЖИЛИЩНИК РАЙОНА ТВЕРСКОЙ»,Центральный,431
6,«ЖИЛИЩНИК РАЙОНА ХОРОШЕВО-МНЕВНИКИ»,Северо-Западный,426
7,«ЖИЛИЩНИК РАЙОНА ЛЮБЛИНО»,Юго-Восточный,421
8,«ЖИЛИЩНИК МОЖАЙСКОГО РАЙОНА»,Западный,388
9,«ЖИЛИЩНИК РАЙОНА ЮЖНОЕ БУТОВО»,Юго-Западный,379


## Промежуточный вывод

В рейтинге за 2019 год представлено 562 записи, за 2020 год - 605 записей, за 2021 год - 622 записи.

Все записи в каждом из рейтингов уникальны.

В оценках компаний внутри административных окургов нет выбросов.

In [23]:
# Сохранение датафрейма
rate_2019.to_csv('data/rate_2019.csv', index=False)

rate_2020.to_csv('data/rate_2020.csv', index=False)

rate_2021.to_csv('data/rate_2021.csv', index=False)